# Cased Final Bert Implementations

Use Columbia Account as data is stored there

In [0]:
!pip3 install http://download.pytorch.org/whl/cu80/torch-0.3.0.post4-cp36-cp36m-linux_x86_64.whl 
!pip3 install torchvision
!pip install pytorch-pretrained-bert
!pip install transformers
!pip install tools
!pip install import-ipynb

     |████████████████████████████████| 592.3MB 55.6MB/s 
ERROR: torchvision 0.4.2 has requirement torch==1.3.1, but you'll have torch 0.3.0.post4 which is incompatible.
ERROR: fastai 1.0.59 has requirement torch>=1.0.0, but you'll have torch 0.3.0.post4 which is incompatible.
  Found existing installation: torch 1.3.1
    Uninstalling torch-1.3.1:
      Successfully uninstalled torch-1.3.1
     |████████████████████████████████| 734.6MB 23kB/s 
  Found existing installation: torch 0.3.0.post4
    Uninstalling torch-0.3.0.post4:
      Successfully uninstalled torch-0.3.0.post4
     |████████████████████████████████| 133kB 264kB/s 
     |████████████████████████████████| 645kB 1.2MB/s 
     |████████████████████████████████| 317kB 519kB/s 
     |████████████████████████████████| 860kB 49.7MB/s 
     |████████████████████████████████| 1.0MB 38.7MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.35-cp36-none-any.whl size=883999 sha256=79f02120a9a1fda38b00b736cb3caf230c3f92aaf9a

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import torch
from transformers import *
import numpy as np

In [0]:
# Old Train Test

# import pandas as pd 
# file = "/content/drive/My Drive/ameya_data/"
# coll_file = "/content/drive/My Drive/Bloomberg Editorial Classifier/data/collected/"

In [0]:
# Old Train Test

# train_df = pd.read_csv(coll_file + "combined_data.csv")
# train_df["label_val"] = train_df["label"].apply(lambda x: 1 if x == "regular" else 0)
# train_df = train_df.dropna()
# test_df = pd.read_csv(coll_file + "combined_data_sample.csv")
# test_df["label_val"] = test_df["label"].apply(lambda x: 1 if x == "regular" else 0)
# test_df = test_df.dropna()

# train_df_bert = pd.DataFrame({
#     'id':range(len(train_df)),
#     'label':train_df["label_val"],
#     'alpha':['a']*train_df.shape[0],
#     'text': train_df["article_text"].replace(r'\n', ' ', regex=True)
# })

# train_df_bert.head()


# dev_df_bert = pd.DataFrame({
#     'id':range(len(test_df)),
#     'label':test_df["label_val"],
#     'alpha':['a']*test_df.shape[0],
#     'text': test_df["article_text"].replace(r'\n', ' ', regex=True)
# })

# dev_df_bert.head()


## Training step



In [0]:
import pandas as pd 
file = "/content/drive/My Drive/ameya_data/"
ttcoll_file = "/content/drive/My Drive/Bloomberg Editorial Classifier/data/train_test_v2/"

In [0]:
import pickle
pkl_file = open(ttcoll_file+ 'X_train.pkl', 'rb')
x_train = pickle.load(pkl_file)
pkl_file.close()

pkl_file = open(ttcoll_file+ 'X_test.pkl', 'rb')
x_test = pickle.load(pkl_file)
pkl_file.close()

pkl_file = open(ttcoll_file+ 'y_train.pkl', 'rb')
y_train = pickle.load(pkl_file)
pkl_file.close()

pkl_file = open(ttcoll_file+ 'y_test.pkl', 'rb')
y_test = pickle.load(pkl_file)
pkl_file.close()

In [0]:
# Temporary fix for Press Democrat
x_train = x_train.drop(5983) 
y_train = y_train.drop(5983)
x_test = x_test.drop([2878,3103,3113,3350,3395,3476,3494])
y_test = y_test.drop([2878,3103,3113,3350,3395,3476,3494])

In [0]:
train_df_bert = pd.DataFrame({
    'id':range(len(x_train)),
    'label':y_train["label"],
    'alpha':['a']*x_train.shape[0],
    'text': x_train["article_text"].replace(r'\n', ' ', regex=True)
})

train_df_bert

,id,label,alpha,text
0,0,1,a,News flash: There's a crisis at the border. T...
1,1,1,a,As I read about several politicians talking a...
2,2,1,a,Look around Bakersfield. At nearly every majo...
3,3,1,a,Our community’s struggle with people roaming ...
4,4,1,a,With their ruling absolving themselves of res...
...,...,...,...,...
6381,6380,0,a,"FAIRBANKS, Alaska — It’s not unusual for big-t..."
6382,6381,0,a,"Judging by his tweets, Dez Bryant has a lot of..."
6383,6382,0,a,Minnesota Timberwolves guard Tyus Jones was st...
6384,6383,0,a,Attorney General Jeff Sessions resigned on Wed...


In [0]:
dev_df_bert = pd.DataFrame({
    'id':range(len(x_train)+ 1 ,len(x_train) + len(x_test) + 1), #
    'label':y_test["label"],
    'alpha':['a']*x_test.shape[0],
    'text': x_test["article_text"].replace(r'\n', ' ', regex=True)
})

dev_df_bert


,id,label,alpha,text
21209,6386,0,a,A cheeseburger is just another slice of Ameri...
21213,6387,0,a,The Kern County Board of Supervisors finally ...
21226,6388,0,a,Today is Opening Day: Gates open at 3 p.m. Ho...
21227,6389,1,a,"Something wicked this way comes, but it’s not..."
21228,6390,0,a,"LONDON - September 18, 2019 - ( Newswire.com ..."
...,...,...,...,...
19579,9810,0,a,President Trump’s frequent characterizations o...
19580,9811,0,a,The Greene County Chamber of Commerce will hos...
19581,9812,1,a,Say ‘no’ to legalizing recreational marijuana ...
19582,9813,0,a,Japan will be at the center of the fall flower...


In [0]:
# Save to Tab Seperated format. Bert Takes TSV for processing

train_df_bert.to_csv(file + "train.tsv", sep='\t', index=False, header=False)
dev_df_bert.to_csv(file + 'dev.tsv', sep='\t', index=False, header=False)

In [0]:
import torch
import pickle
from torch.utils.data import (DataLoader, RandomSampler, SequentialSampler, TensorDataset)
from torch.nn import CrossEntropyLoss, MSELoss
from tqdm import tqdm_notebook, trange
import os
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM, BertForSequenceClassification
from pytorch_pretrained_bert.optimization import BertAdam, WarmupLinearSchedule
from multiprocessing import Pool, cpu_count
from tools import *

import logging
logging.basicConfig(level=logging.INFO)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [0]:
## Variables

# The input data directory containing the tsv
DATA_DIR = file

# Bert Model Selected
# bert-large-uncased, bert-base-cased, bert-large-cased, bert-base-multilingual-uncased,
# bert-base-multilingual-cased, bert-base-chinese.
BERT_MODEL = 'bert-base-cased'

TASK_NAME = 'bloomberg'

# The output directory where the fine-tuned model and checkpoints will be written.
OUTPUT_DIR = f'outputs/{TASK_NAME}/'

# The directory where the evaluation reports will be written to.
REPORTS_DIR = f'reports/{TASK_NAME}_evaluation_report/'

# This is where BERT will look for pre-trained models to load parameters from.
CACHE_DIR = 'cache/'

# Length of input sequence for input
MAX_SEQ_LENGTH = 200

TRAIN_BATCH_SIZE = 24
EVAL_BATCH_SIZE = 24
LEARNING_RATE = 2e-5
NUM_TRAIN_EPOCHS =10
RANDOM_SEED = 42
GRADIENT_ACCUMULATION_STEPS = 1
WARMUP_PROPORTION = 0.1

cache_dir = CACHE_DIR

CONFIG_NAME = "config.json"
WEIGHTS_NAME = "pytorch_model.bin"

In [0]:
if os.path.exists(REPORTS_DIR) and os.listdir(REPORTS_DIR):
        REPORTS_DIR += f'/report_{len(os.listdir(REPORTS_DIR))}'
        os.makedirs(REPORTS_DIR)
if not os.path.exists(REPORTS_DIR):
    os.makedirs(REPORTS_DIR)
    REPORTS_DIR += f'/report_{len(os.listdir(REPORTS_DIR))}'
    os.makedirs(REPORTS_DIR)

In [0]:
import shutil
if os.path.exists(OUTPUT_DIR) and os.listdir(OUTPUT_DIR):
        shutil.rmtree(OUTPUT_DIR)
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

In [0]:

from __future__ import absolute_import, division, print_function

import csv
import os
import sys
import logging

logger = logging.getLogger()
csv.field_size_limit(2147483647) # Increase CSV reader's field limit incase we have long text.


class InputExample(object):
    """A single training/test example for simple sequence classification."""

    def __init__(self, guid, text_a, text_b=None, label=None):
        """Constructs a InputExample.
        Args:
            guid: Unique id for the example.
            text_a: string. The untokenized text of the first sequence. For single
            sequence tasks, only this sequence must be specified.
            text_b: (Optional) string. The untokenized text of the second sequence.
            Only must be specified for sequence pair tasks.
            label: (Optional) string. The label of the example. This should be
            specified for train and dev examples, but not for test examples.
        """
        self.guid = guid
        self.text_a = text_a
        self.text_b = text_b
        self.label = label


class DataProcessor(object):
    """Base class for data converters for sequence classification data sets."""

    def get_train_examples(self, data_dir):
        """Gets a collection of `InputExample`s for the train set."""
        raise NotImplementedError()

    def get_dev_examples(self, data_dir):
        """Gets a collection of `InputExample`s for the dev set."""
        raise NotImplementedError()

    def get_labels(self):
        """Gets the list of labels for this data set."""
        raise NotImplementedError()

    @classmethod
    def _read_tsv(cls, input_file, quotechar=None):
        """Reads a tab separated value file."""
        with open(input_file, "r", encoding="utf-8") as f:
            reader = csv.reader(f, delimiter="\t", quotechar=quotechar)
            lines = []
            for line in reader:
                if sys.version_info[0] == 2:
                    line = list(unicode(cell, 'utf-8') for cell in line)
                lines.append(line)
            return lines


class BinaryClassificationProcessor(DataProcessor):
    """Processor for binary classification dataset."""

    def get_train_examples(self, data_dir):
        """See base class."""
        return self._create_examples(
            self._read_tsv(os.path.join(data_dir, "train.tsv")), "train")

    def get_dev_examples(self, data_dir):
        """See base class."""
        return self._create_examples(
            self._read_tsv(os.path.join(data_dir, "dev.tsv")), "dev")

    def get_labels(self):
        """See base class."""
        return ["0", "1"]

    def _create_examples(self, lines, set_type):
        """Creates examples for the training and dev sets."""
        examples = []
        for (i, line) in enumerate(lines):
            guid = "%s-%s" % (set_type, i)
            try:
              text_a = line[3]
              label = line[1]
              examples.append(
                InputExample(guid=guid, text_a=text_a, text_b=None, label=label))
            except:
              pass
        return examples


In [0]:

class InputFeatures(object):
    """A single set of features of data."""

    def __init__(self, input_ids, input_mask, segment_ids, label_id):
        self.input_ids = input_ids
        self.input_mask = input_mask
        self.segment_ids = segment_ids
        self.label_id = label_id


def _truncate_seq_pair(tokens_a, tokens_b, max_length):
    """Truncates a sequence pair in place to the maximum length."""

    # This is a simple heuristic which will always truncate the longer sequence
    # one token at a time. This makes more sense than truncating an equal percent
    # of tokens from each, since if one sequence is very short then each token
    # that's truncated likely contains more information than a longer sequence.
    while True:
        total_length = len(tokens_a) + len(tokens_b)
        if total_length <= max_length:
            break
        if len(tokens_a) > len(tokens_b):
            tokens_a.pop()
        else:
            tokens_b.pop()




def convert_example_to_feature(example_row):
  # return example_row
  example, label_map, max_seq_length, tokenizer = example_row

  tokens_a = tokenizer.tokenize(example.text_a)

  tokens_b = None
  if example.text_b:
      tokens_b = tokenizer.tokenize(example.text_b)
      # Modifies `tokens_a` and `tokens_b` in place so that the total
      # length is less than the specified length.
      # Account for [CLS], [SEP], [SEP] with "- 3"
      _truncate_seq_pair(tokens_a, tokens_b, max_seq_length - 3)
  else:
      # Account for [CLS] and [SEP] with "- 2"
      if len(tokens_a) > max_seq_length - 2:
          tokens_a = tokens_a[:(max_seq_length - 2)]

  tokens = ["[CLS]"] + tokens_a + ["[SEP]"]
  segment_ids = [0] * len(tokens)

  if tokens_b:
      tokens += tokens_b + ["[SEP]"]
      segment_ids += [1] * (len(tokens_b) + 1)

  input_ids = tokenizer.convert_tokens_to_ids(tokens)

  # The mask has 1 for real tokens and 0 for padding tokens. Only real
  # tokens are attended to.
  input_mask = [1] * len(input_ids)

  # Zero-pad up to the sequence length.
  padding = [0] * (max_seq_length - len(input_ids))
  input_ids += padding
  input_mask += padding
  segment_ids += padding

  assert len(input_ids) == max_seq_length
  assert len(input_mask) == max_seq_length
  assert len(segment_ids) == max_seq_length

  label_id = label_map[example.label]


  return InputFeatures(input_ids=input_ids,
                       input_mask=input_mask,
                       segment_ids=segment_ids,
                       label_id=label_id)



In [0]:
# Binary Classification

processor = BinaryClassificationProcessor()
train_examples = processor.get_train_examples(DATA_DIR)
train_examples_len = len(train_examples)
label_list = processor.get_labels() # [0, 1] for binary classification
num_labels = len(label_list)
num_train_optimization_steps = int(
    train_examples_len / TRAIN_BATCH_SIZE / GRADIENT_ACCUMULATION_STEPS) * NUM_TRAIN_EPOCHS
# Load pre-trained model tokenizer (vocabulary)
tokenizer = BertTokenizer.from_pretrained('bert-base-cased', do_lower_case=False)

INFO:pytorch_pretrained_bert.file_utils:https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-cased-vocab.txt not found in cache, downloading to /tmp/tmpaedvhyr0
100%|██████████| 213450/213450 [00:00<00:00, 856466.82B/s]
INFO:pytorch_pretrained_bert.file_utils:copying /tmp/tmpaedvhyr0 to cache at /root/.pytorch_pretrained_bert/5e8a2b4893d13790ed4150ca1906be5f7a03d6c4ddf62296c383f6db42814db2.e13dbb970cb325137104fb2e5f36fe865f27746c6b526f6352861b1980eb80b1
INFO:pytorch_pretrained_bert.file_utils:creating metadata file for /root/.pytorch_pretrained_bert/5e8a2b4893d13790ed4150ca1906be5f7a03d6c4ddf62296c383f6db42814db2.e13dbb970cb325137104fb2e5f36fe865f27746c6b526f6352861b1980eb80b1
INFO:pytorch_pretrained_bert.file_utils:removing temp file /tmp/tmpaedvhyr0
INFO:pytorch_pretrained_bert.tokenization:loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-cased-vocab.txt from cache at /root/.pytorch_pretrained_bert/5e8a2b4893d13790ed4150ca1906be5f7a03d6c4

In [0]:
label_map = {label: i for i, label in enumerate(label_list)}
train_examples_for_processing = [(example, label_map, MAX_SEQ_LENGTH, tokenizer) for example in train_examples]

In [0]:
# Featuring of words
process_count = cpu_count() - 1
if __name__ ==  '__main__':
    print(f'Preparing to convert {train_examples_len} examples..')
    print(f'Spawning {process_count} processes..')
    with Pool(process_count) as p:
        train_features = list(tqdm_notebook(p.imap(convert_example_to_feature, train_examples_for_processing), total=train_examples_len))

Preparing to convert 6385 examples..
Spawning 1 processes..


In [0]:
# with open(file + "train_features.pkl", 'wb') as f:
#     pickle.dump(train_features, f)
# with open(file + "train_features.pkl", 'rb') as f:
#     train_features = pickle.load(f)

In [0]:
# Load pre-trained model (weights)
model = BertForSequenceClassification.from_pretrained(BERT_MODEL, cache_dir=CACHE_DIR, num_labels=num_labels)


INFO:pytorch_pretrained_bert.file_utils:https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-cased.tar.gz not found in cache, downloading to /tmp/tmprd7o7g2p
100%|██████████| 404400730/404400730 [00:16<00:00, 24469221.61B/s]
INFO:pytorch_pretrained_bert.file_utils:copying /tmp/tmprd7o7g2p to cache at cache/a803ce83ca27fecf74c355673c434e51c265fb8a3e0e57ac62a80e38ba98d384.681017f415dfb33ec8d0e04fe51a619f3f01532ecea04edbfd48c5d160550d9c
INFO:pytorch_pretrained_bert.file_utils:creating metadata file for cache/a803ce83ca27fecf74c355673c434e51c265fb8a3e0e57ac62a80e38ba98d384.681017f415dfb33ec8d0e04fe51a619f3f01532ecea04edbfd48c5d160550d9c
INFO:pytorch_pretrained_bert.file_utils:removing temp file /tmp/tmprd7o7g2p
INFO:pytorch_pretrained_bert.modeling:loading archive file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-cased.tar.gz from cache at cache/a803ce83ca27fecf74c355673c434e51c265fb8a3e0e57ac62a80e38ba98d384.681017f415dfb33ec8d0e04fe51a619f3f01532ecea04edbfd48c5d

In [0]:

model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): BertLayerNorm()
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): BertLayerNorm()
              (dropout): Dropout(p=0.1, inplace=False)
   

In [0]:
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
    ]
optimizer = BertAdam(optimizer_grouped_parameters,
                     lr=LEARNING_RATE,
                     warmup=WARMUP_PROPORTION,
                     t_total=num_train_optimization_steps)
global_step = 0
nb_tr_steps = 0
tr_loss = 0
logger.info("***** Running training *****")
logger.info("  Num examples = %d", train_examples_len)
logger.info("  Batch size = %d", TRAIN_BATCH_SIZE)
logger.info("  Num steps = %d", num_train_optimization_steps)
all_input_ids = torch.tensor([f.input_ids for f in train_features], dtype=torch.long)
all_input_mask = torch.tensor([f.input_mask for f in train_features], dtype=torch.long)
all_segment_ids = torch.tensor([f.segment_ids for f in train_features], dtype=torch.long)

all_label_ids = torch.tensor([f.label_id for f in train_features], dtype=torch.long)


INFO:root:***** Running training *****
INFO:root:  Num examples = 6385
INFO:root:  Batch size = 24
INFO:root:  Num steps = 2660


In [0]:
train_data = TensorDataset(all_input_ids, all_input_mask, all_segment_ids, all_label_ids)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=TRAIN_BATCH_SIZE)

In [0]:
model.train()
for _ in trange(int(NUM_TRAIN_EPOCHS), desc="Epoch"):
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0
    for step, batch in enumerate(tqdm_notebook(train_dataloader, desc="Iteration")):
        batch = tuple(t.to(device) for t in batch)
        input_ids, input_mask, segment_ids, label_ids = batch

        logits = model(input_ids, segment_ids, input_mask, labels=None)

        loss_fct = CrossEntropyLoss()
        loss = loss_fct(logits.view(-1, num_labels), label_ids.view(-1))
        

        if GRADIENT_ACCUMULATION_STEPS > 1:
            loss = loss / GRADIENT_ACCUMULATION_STEPS

        loss.backward()
        print("\r%f" % loss, end='')
        
        tr_loss += loss.item()
        nb_tr_examples += input_ids.size(0)
        nb_tr_steps += 1
        if (step + 1) % GRADIENT_ACCUMULATION_STEPS == 0:
            optimizer.step()
            optimizer.zero_grad()
            global_step += 1

Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

0.203721

Epoch:  10%|█         | 1/10 [08:29<1:16:27, 509.67s/it]

0.130288


0.287632

...

0.000550

0.000629

0.000477

Epoch: 100%|██████████| 10/10 [1:25:07<00:00, 512.53s/it]

0.000103


In [0]:

model_to_save = model.module if hasattr(model, 'module') else model  # Only save the model it-self

# If we save using the predefined names, we can load using `from_pretrained`
output_model_file = os.path.join(OUTPUT_DIR, WEIGHTS_NAME)
output_config_file = os.path.join(OUTPUT_DIR, CONFIG_NAME)

torch.save(model_to_save.state_dict(), output_model_file)
model_to_save.config.to_json_file(output_config_file)
tokenizer.save_vocabulary(OUTPUT_DIR)

'outputs/bloomberg/vocab.txt'

##Evaluation setup

This is to save the files for the evaluation setup

In [0]:
import tarfile

def make_tarfile(output_filename, source_dirs):
    with tarfile.open(output_filename, "w:gz") as tar:
      for source_dir in source_dirs:
        tar.add(source_dir, arcname=os.path.basename(source_dir))

In [0]:
make_tarfile("/content/cache/bloomberg.tar.gz", ["/content/outputs/bloomberg/config.json", "/content/outputs/bloomberg/pytorch_model.bin", "/content/outputs/bloomberg/vocab.txt"])

## Evaluation

In [0]:
from sklearn.metrics import confusion_matrix
from torch.utils.data.distributed import DistributedSampler
BERT_MODEL = 'bloomberg.tar.gz'
REPORTS_DIR = f'reports/{TASK_NAME}_evaluation_reports/'

if os.path.exists(REPORTS_DIR) and os.listdir(REPORTS_DIR):
        REPORTS_DIR += f'/report_{len(os.listdir(REPORTS_DIR))}'
        os.makedirs(REPORTS_DIR)
if not os.path.exists(REPORTS_DIR):
    os.makedirs(REPORTS_DIR)
    REPORTS_DIR += f'/report_{len(os.listdir(REPORTS_DIR))}'
    os.makedirs(REPORTS_DIR)
    

In [0]:
# Load pre-trained model tokenizer (vocabulary)
tokenizer = BertTokenizer.from_pretrained(OUTPUT_DIR + 'vocab.txt', do_lower_case=False)



INFO:pytorch_pretrained_bert.tokenization:loading vocabulary file outputs/bloomberg/vocab.txt


In [0]:
processor = BinaryClassificationProcessor()
eval_examples = processor.get_dev_examples(DATA_DIR)
label_list = processor.get_labels() # [0, 1] for binary classification
num_labels = len(label_list)
eval_examples_len = len(eval_examples)


In [0]:
label_map = {label: i for i, label in enumerate(label_list)}
eval_examples_for_processing = [(example, label_map, MAX_SEQ_LENGTH, tokenizer) for example in eval_examples]


In [0]:
process_count = cpu_count() - 1
if __name__ ==  '__main__':
    print(f'Preparing to convert {eval_examples_len} examples..')
    print(f'Spawning {process_count} processes..')
    with Pool(process_count) as p:
        eval_features = list(tqdm_notebook(p.imap(convert_example_to_feature, eval_examples_for_processing), total=eval_examples_len))

Preparing to convert 3429 examples..
Spawning 1 processes..


In [0]:

all_input_ids = torch.tensor([f.input_ids for f in eval_features], dtype=torch.long)
all_input_mask = torch.tensor([f.input_mask for f in eval_features], dtype=torch.long)
all_segment_ids = torch.tensor([f.segment_ids for f in eval_features], dtype=torch.long)

In [0]:
all_label_ids = torch.tensor([f.label_id for f in eval_features], dtype=torch.long)


In [0]:

eval_data = TensorDataset(all_input_ids, all_input_mask, all_segment_ids, all_label_ids)

# Run prediction for full data
eval_sampler = SequentialSampler(eval_data)
eval_dataloader = DataLoader(eval_data, sampler=eval_sampler, batch_size=EVAL_BATCH_SIZE)

In [0]:
BERT_MODEL

'bloomberg.tar.gz'

In [0]:

# Load pre-trained model (weights)
model = BertForSequenceClassification.from_pretrained(CACHE_DIR + BERT_MODEL, cache_dir=CACHE_DIR, num_labels=len(label_list))

INFO:pytorch_pretrained_bert.modeling:loading archive file cache/bloomberg.tar.gz
INFO:pytorch_pretrained_bert.modeling:extracting archive file cache/bloomberg.tar.gz to temp dir /tmp/tmpmyhsx7_7
INFO:pytorch_pretrained_bert.modeling:Model config {
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "type_vocab_size": 2,
  "vocab_size": 28996
}



In [0]:
model.to(device)


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): BertLayerNorm()
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): BertLayerNorm()
              (dropout): Dropout(p=0.1, inplace=False)
   

In [0]:
model.eval()
eval_loss = 0
nb_eval_steps = 0
preds = []

for input_ids, input_mask, segment_ids, label_ids in tqdm_notebook(eval_dataloader, desc="Evaluating"):
    input_ids = input_ids.to(device)
    input_mask = input_mask.to(device)
    segment_ids = segment_ids.to(device)
    label_ids = label_ids.to(device)

    with torch.no_grad():
        logits = model(input_ids, segment_ids, input_mask, labels=None)

    # create eval loss and other metric required by the task
    loss_fct = CrossEntropyLoss()
    tmp_eval_loss = loss_fct(logits.view(-1, num_labels), label_ids.view(-1))
    

    eval_loss += tmp_eval_loss.mean().item()
    nb_eval_steps += 1
    if len(preds) == 0:
        preds.append(logits.detach().cpu().numpy())
    else:
        preds[0] = np.append(
            preds[0], logits.detach().cpu().numpy(), axis=0)



In [0]:
def get_eval_report(labels, preds):
    tn, fp, fn, tp = confusion_matrix(labels, preds).ravel()
    pre = tp / (tp + fp)
    rec = tp / (tp + fn)
    tnr = tn / (tn + fp)
    acc = (tp + tn)/ (tp + tn + fp + fn)
    bacc = (rec + tnr) / 2
    f1 = 2 * (pre * rec)/(pre+rec)
    return {
        "tp": tp,
        "tn": tn,
        "fp": fp,
        "fn": fn,
        "Precision" : pre,
        "Recall/TPR" : rec,
        "TNR" : tnr,
        "Accuracy" : acc,
        "Balanced Accuracy" : bacc,
        "F1 Score" : f1,
    }

def compute_metrics(labels, preds):
    assert len(preds) == len(labels)
    return get_eval_report(labels, preds)

In [0]:
predictions = preds[0]
predictions = np.argmax(predictions, axis=1)
result = compute_metrics(all_label_ids.numpy(), predictions)
output_eval_file = os.path.join(REPORTS_DIR, "eval_results.txt")
with open(output_eval_file, "w") as writer:
    logger.info("***** Eval results *****")
    for key in (result.keys()):
        logger.info("  %s = %s", key, str(result[key]))
        writer.write("%s = %s\n" % (key, str(result[key])))

INFO:root:***** Eval results *****
INFO:root:  tp = 325
INFO:root:  tn = 2876
INFO:root:  fp = 200
INFO:root:  fn = 28
INFO:root:  Precision = 0.6190476190476191
INFO:root:  Recall/TPR = 0.9206798866855525
INFO:root:  TNR = 0.9349804941482445
INFO:root:  Accuracy = 0.9335083114610674
INFO:root:  Balanced Accuracy = 0.9278301904168984
INFO:root:  F1 Score = 0.7403189066059227
